In [ ]:
"""
Pyroshell Market Sizing — Figures Pack (US + Canada)
All revenue charts use USD (millions). Saves to OUT_DIR.
"""

from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from matplotlib.patches import Rectangle, ConnectionPatch

# =========================
# PARAMETERS (edit as needed)
# =========================
# Unit counts
US_SITES = 450_000
CA_SITES = 56_700

# Exposure assumptions
US_EXPOSURE = 0.33
CA_EXPOSURE_BASE = 0.20   # base (you may add low/high in your doc text)

# Price & service
ASP = 4_500               # $/site installed
SERVICE_ATTACH = 0.40
SERVICE_FEE = 350

# Phase-1 footprint (SAM) & Gen-1 fit
US_WEST_SHARE = 0.55
CA_WEST_SHARE = 0.50
GEN1_FIT = 0.80

# SOM capacity & ramp (shared NA crew pool split 80/20)
NA_CREWS = 6
INSTALLS_PER_WEEK = 4
WEEKS_PER_YEAR = 45
ADOPTION_RAMP = [0.50, 0.85, 1.00]
US_CAPACITY_SHARE = 0.80
CA_CAPACITY_SHARE = 0.20

# Output folder
OUT_DIR = Path("image")  # change to Path(r"D:\Repository\mech_431\image") if you want absolute
OUT_DIR.mkdir(parents=True, exist_ok=True)

# =========================
# COLORS
# =========================
C_US = "#0A3161"      # blue for United States
C_CA = "#EF3340"      # red for Canada
EDGE = "#333333"      # outline
C_US_light = "#2F5F9E"
C_CA_light = "#F36A73"
C_TAM = "#A6CEE3"
C_SAM = "#1F78B4"
C_SOM = "#0B4F79"

# =========================
# HELPERS
# =========================
def som_over_3yrs(sam_assets: float, cap_per_year: float, ramp):
    installs = []
    remaining = float(sam_assets)
    for r in ramp:
        x = min(r * cap_per_year, max(remaining, 0))
        installs.append(x)
        remaining -= x
    return sum(installs), installs

def add_value_labels(ax, xs, ys, fmt):
    for x, y in zip(xs, ys):
        ax.text(x, y, fmt(y), ha="center", va="bottom")

def fmt_int(v):      # integer with commas
    return f"{int(round(v)):,}"

def fmt_money_M(v):  # value already in millions
    return f"${v:.1f}M"

def bar_simple(labels, values, colors, title, ylabel, out_name, value_fmt):
    fig, ax = plt.subplots(figsize=(7.5, 5))
    xs = np.arange(len(labels))
    ax.bar(labels, values, color=colors, edgecolor=EDGE)
    add_value_labels(ax, xs, values, value_fmt)
    ax.set_ylabel(ylabel)
    ax.set_title(title)
    plt.tight_layout()
    p = OUT_DIR / out_name
    plt.savefig(p, dpi=300)
    plt.close()
    print(f"[Saved] {p.resolve()}")

# =========================
# CORE CALCS
# =========================
# TAM assets
US_TAM_assets = US_SITES * US_EXPOSURE
CA_TAM_assets = CA_SITES * CA_EXPOSURE_BASE

# SAM assets
US_SAM_assets = US_TAM_assets * US_WEST_SHARE * GEN1_FIT
CA_SAM_assets = CA_TAM_assets * CA_WEST_SHARE * GEN1_FIT

# SOM capacity & installs
capacity_total = NA_CREWS * INSTALLS_PER_WEEK * WEEKS_PER_YEAR
US_capacity = capacity_total * US_CAPACITY_SHARE
CA_capacity = capacity_total * CA_CAPACITY_SHARE

US_SOM_assets, US_installs_by_year = som_over_3yrs(US_SAM_assets, US_capacity, ADOPTION_RAMP)
CA_SOM_assets, CA_installs_by_year = som_over_3yrs(CA_SAM_assets, CA_capacity, ADOPTION_RAMP)

# One-time revenue ($)
US_TAM_rev = US_TAM_assets * ASP
CA_TAM_rev = CA_TAM_assets * ASP
US_SAM_rev = US_SAM_assets * ASP
CA_SAM_rev = CA_SAM_assets * ASP
US_SOM_rev = US_SOM_assets * ASP
CA_SOM_rev = CA_SOM_assets * ASP

# Combined
COMB_TAM_assets = US_TAM_assets + CA_TAM_assets
COMB_SAM_assets = US_SAM_assets + CA_SAM_assets
COMB_SOM_assets = US_SOM_assets + CA_SOM_assets
COMB_TAM_rev = US_TAM_rev + CA_TAM_rev
COMB_SAM_rev = US_SAM_rev + CA_SAM_rev
COMB_SOM_rev = US_SOM_rev + CA_SOM_rev

# =========================
# 1) US — TAM/SAM/SOM (assets)
# =========================
bar_simple(
    labels=["TAM", "SAM", "SOM"],
    values=[US_TAM_assets, US_SAM_assets, US_SOM_assets],
    colors=[C_TAM, C_SAM, C_SOM],
    title="United States — TAM / SAM / SOM (Assets)",
    ylabel="Number of assets (sites)",
    out_name="fig_US_TAM_SAM_SOM_assets.png",
    value_fmt=fmt_int
)

# =========================
# 2) Canada — TAM/SAM/SOM (assets)
# =========================
bar_simple(
    labels=["TAM", "SAM", "SOM"],
    values=[CA_TAM_assets, CA_SAM_assets, CA_SOM_assets],
    colors=[C_TAM, C_SAM, C_SOM],
    title="Canada (Base) — TAM / SAM / SOM (Assets)",
    ylabel="Number of assets (sites)",
    out_name="fig_CA_TAM_SAM_SOM_assets.png",
    value_fmt=fmt_int
)

# =========================
# 3) Combined (assets), stacked US + Canada for each tier
# =========================
fig, ax = plt.subplots(figsize=(7.8, 5.2))
x = np.arange(3)  # TAM, SAM, SOM
us_vals = np.array([US_TAM_assets, US_SAM_assets, US_SOM_assets], dtype=float)
ca_vals = np.array([CA_TAM_assets, CA_SAM_assets, CA_SOM_assets], dtype=float)

ax.bar(x, us_vals, color=C_US, edgecolor=EDGE, label="United States")
ax.bar(x, ca_vals, bottom=us_vals, color=C_CA, edgecolor=EDGE, label="Canada (base)")
add_value_labels(ax, x, us_vals + ca_vals, fmt_int)

ax.set_xticks(x); ax.set_xticklabels(["TAM", "SAM", "SOM"])
ax.set_ylabel("Number of assets (sites)")
ax.set_title("Assets Summary")
ax.legend()
plt.tight_layout()
p = OUT_DIR / "fig_combined_TAM_SAM_SOM_assets_stacked.png"
plt.savefig(p, dpi=300); plt.close()
print(f"[Saved] {p.resolve()}")

# =========================
# 4) Combined (one-time $), stacked US + Canada — USD millions
# =========================
fig, ax = plt.subplots(figsize=(7.8, 5.2))
us_rev_M = np.array([US_TAM_rev, US_SAM_rev, US_SOM_rev], dtype=float) / 1e6
ca_rev_M = np.array([CA_TAM_rev, CA_SAM_rev, CA_SOM_rev], dtype=float) / 1e6

ax.bar(x, us_rev_M, color=C_US, edgecolor=EDGE, label="United States")
ax.bar(x, ca_rev_M, bottom=us_rev_M, color=C_CA, edgecolor=EDGE, label="Canada (base)")
add_value_labels(ax, x, us_rev_M + ca_rev_M, fmt_money_M)

ax.set_xticks(x); ax.set_xticklabels(["TAM", "SAM", "SOM"])
ax.set_ylabel("USD (millions)")
ax.set_title("One-time Revenue Summary")
ax.legend()
plt.tight_layout()
p = OUT_DIR / "fig_combined_TAM_SAM_SOM_revenue_stacked.png"
plt.savefig(p, dpi=300); plt.close()
print(f"[Saved] {p.resolve()}")

# =========================
# 5–7) US vs Canada — TAM/SAM/SOM (assets)
# =========================
bar_simple(
    labels=["United States", "Canada (base)"],
    values=[US_TAM_assets, CA_TAM_assets],
    colors=[C_US, C_CA],
    title="TAM (Assets)",
    ylabel="Number of assets (sites)",
    out_name="fig_compare_US_vs_CA_TAM_assets.png",
    value_fmt=fmt_int
)
bar_simple(
    labels=["United States", "Canada (base)"],
    values=[US_SAM_assets, CA_SAM_assets],
    colors=[C_US, C_CA],
    title="SAM (Assets)",
    ylabel="Number of assets (sites)",
    out_name="fig_compare_US_vs_CA_SAM_assets.png",
    value_fmt=fmt_int
)
bar_simple(
    labels=["United States", "Canada (base)"],
    values=[US_SOM_assets, CA_SOM_assets],
    colors=[C_US, C_CA],
    title="SOM (Assets, 3-yr)",
    ylabel="Number of assets (sites)",
    out_name="fig_compare_US_vs_CA_SOM_assets.png",
    value_fmt=fmt_int
)

# =========================
# 8–10) US vs Canada — TAM/SAM/SOM (one-time $) — USD millions
# =========================
bar_simple(
    labels=["United States", "Canada (base)"],
    values=[US_TAM_rev/1e6, CA_TAM_rev/1e6],
    colors=[C_US, C_CA],
    title="TAM (One-time $)",
    ylabel="USD (millions)",
    out_name="fig_compare_US_vs_CA_TAM_revenue.png",
    value_fmt=fmt_money_M
)
bar_simple(
    labels=["United States", "Canada (base)"],
    values=[US_SAM_rev/1e6, CA_SAM_rev/1e6],
    colors=[C_US, C_CA],
    title="SAM (One-time $)",
    ylabel="USD (millions)",
    out_name="fig_compare_US_vs_CA_SAM_revenue.png",
    value_fmt=fmt_money_M
)
bar_simple(
    labels=["United States", "Canada (base)"],
    values=[US_SOM_rev/1e6, CA_SOM_rev/1e6],
    colors=[C_US, C_CA],
    title="SOM (One-time $, 3-yr)",
    ylabel="USD (millions)",
    out_name="fig_compare_US_vs_CA_SOM_revenue.png",
    value_fmt=fmt_money_M
)

# =========================
# 11) Sensitivity — U.S. TAM (one-time $) vs Exposure % — USD millions
# =========================
exp_range = np.linspace(0.25, 0.40, 16)  # 25% to 40%
us_tam_rev_sens_M = (US_SITES * exp_range * ASP) / 1e6
fig, ax = plt.subplots(figsize=(7.5, 5))
ax.plot(exp_range * 100, us_tam_rev_sens_M, color=C_US)
ax.set_xlabel("U.S. exposure (%)")
ax.set_ylabel("U.S. TAM one-time (USD, millions)")
ax.set_title("Sensitivity — U.S. TAM (One-time $) vs Exposure %")
ax.grid(True, alpha=0.2)
plt.tight_layout()
p = OUT_DIR / "fig_sensitivity_US_TAM_vs_exposure.png"
plt.savefig(p, dpi=300); plt.close()
print(f"[Saved] {p.resolve()}")

# =========================
# 12) Sensitivity — SOM (3-yr assets) vs installs/week per crew
# =========================
crew_week_range = np.arange(2, 8+1)
som_totals = []
for iw in crew_week_range:
    cap_total = NA_CREWS * iw * WEEKS_PER_YEAR
    us_cap = cap_total * US_CAPACITY_SHARE
    ca_cap = cap_total * CA_CAPACITY_SHARE
    us_som, _ = som_over_3yrs(US_SAM_assets, us_cap, ADOPTION_RAMP)
    ca_som, _ = som_over_3yrs(CA_SAM_assets, ca_cap, ADOPTION_RAMP)
    som_totals.append(us_som + ca_som)

fig, ax = plt.subplots(figsize=(7.5, 5))
ax.plot(crew_week_range, som_totals, marker="o", color=C_SOM)
ax.set_xlabel("Installs per week per crew")
ax.set_ylabel("SOM (3-yr installs, assets)")
ax.set_title("Sensitivity — SOM (3-yr assets) vs Crew Throughput")
ax.grid(True, alpha=0.2)
plt.tight_layout()
p = OUT_DIR / "fig_sensitivity_SOM_vs_installs_per_week.png"
plt.savefig(p, dpi=300); plt.close()
print(f"[Saved] {p.resolve()}")

# =========================
# EXTRA — One figure: TAM, SAM, SOM, Subscription (USD millions) with ONE zoom frame over SOM+Subscription
# =========================
# 3-yr cumulative subscription revenue split US/CA (in millions)
US_active_subs = np.cumsum(np.array(US_installs_by_year)) * SERVICE_ATTACH
CA_active_subs = np.cumsum(np.array(CA_installs_by_year)) * SERVICE_ATTACH
US_subs_rev_total_M = float((US_active_subs * SERVICE_FEE).sum()) / 1e6
CA_subs_rev_total_M = float((CA_active_subs * SERVICE_FEE).sum()) / 1e6

labels4 = ["TAM", "SAM", "SOM", "Subscription"]
us_M = np.array([US_TAM_rev, US_SAM_rev, US_SOM_rev, US_subs_rev_total_M * 1e6], float) / 1e6
ca_M = np.array([CA_TAM_rev, CA_SAM_rev, CA_SOM_rev, CA_subs_rev_total_M * 1e6], float) / 1e6
tot_M = us_M + ca_M

fig, ax = plt.subplots(figsize=(9.2, 5.6))
x4 = np.arange(4)
ax.bar(x4, us_M, color=C_US, edgecolor=EDGE, label="United States")
ax.bar(x4, ca_M, bottom=us_M, color=C_CA, edgecolor=EDGE, label="Canada")
add_value_labels(ax, x4, tot_M, fmt_money_M)
ax.set_xticks(x4); ax.set_xticklabels(labels4)
ax.set_ylabel("USD (millions)")
ax.set_title("One-time & Subscription Revenue (USD millions)")
ax.legend()

# bounding box on main plot (SOM+Subscription region)
x0, x1 = 1.6, 3.4
y0, y1 = 0, max(tot_M[2], tot_M[3]) * 1.15
zoom_box = Rectangle((x0, y0), x1-x0, y1-y0, fill=False, lw=1.2, ec=EDGE)
ax.add_patch(zoom_box)

# single inset (position/size — tweak bbox_to_anchor to move/resize)
axins = inset_axes(
    ax,
    width="34%", height="34%",                 # inset size
    bbox_to_anchor=(0.58, 0.42, 0.34, 0.34),   # (x0,y0,w,h) in axes coords
    bbox_transform=ax.transAxes,
    loc="lower left",
    borderpad=0.0
)
xin = np.array([0, 1])
us_in = np.array([us_M[2], us_M[3]])
ca_in = np.array([ca_M[2], ca_M[3]])
axins.bar(xin, us_in, color=C_US, edgecolor=EDGE)
axins.bar(xin, ca_in, bottom=us_in, color=C_CA, edgecolor=EDGE)
axins.set_xticks(xin); axins.set_xticklabels(["SOM", "Subscription"])
axins.set_ylim(0, max(tot_M[2], tot_M[3]) * 1.25)
axins.tick_params(axis='y', labelsize=8)

# connectors
con1 = ConnectionPatch(xyA=(x0, y1), coordsA=ax.transData, xyB=(0, 0), coordsB=axins.transAxes, lw=1.0, color=EDGE)
con2 = ConnectionPatch(xyA=(x1, y1), coordsA=ax.transData, xyB=(1, 0), coordsB=axins.transAxes, lw=1.0, color=EDGE)
ax.add_artist(con1); ax.add_artist(con2)

plt.tight_layout()
p = OUT_DIR / "fig_TAM_SAM_SOM_SUB_rev_single_zoom_MUSD.png"
plt.savefig(p, dpi=300); plt.close()
print(f"[Saved] {p.resolve()}")

# =========================
# EXTRA — Canada low/base/high grouped charts (Assets + $M)
# =========================
CA_EXPOSURE_LOW  = 0.123
CA_EXPOSURE_HIGH = 0.33

def ca_tiers_assets(exposure):
    tam = CA_SITES * exposure
    sam = tam * CA_WEST_SHARE * GEN1_FIT
    cap_total = NA_CREWS * INSTALLS_PER_WEEK * WEEKS_PER_YEAR
    ca_cap = cap_total * CA_CAPACITY_SHARE
    som, _ = som_over_3yrs(sam, ca_cap, ADOPTION_RAMP)
    return tam, sam, som

tam_L, sam_L, som_L = ca_tiers_assets(CA_EXPOSURE_LOW)
tam_B, sam_B, som_B = ca_tiers_assets(CA_EXPOSURE_BASE)
tam_H, sam_H, som_H = ca_tiers_assets(CA_EXPOSURE_HIGH)

cats = ["TAM", "SAM", "SOM"]
xg = np.arange(len(cats))
w = 0.26
colors = [C_CA_light, C_CA, "#8C1D24"]

# Assets
fig, ax = plt.subplots(figsize=(8.4, 5.2))
ax.bar(xg - w, [tam_L, sam_L, som_L], width=w, color=colors[0], edgecolor=EDGE, label="Low (12.3%)")
ax.bar(xg,      [tam_B, sam_B, som_B], width=w, color=colors[1], edgecolor=EDGE, label="Base (20%)")
ax.bar(xg + w,  [tam_H, sam_H, som_H], width=w, color=colors[2], edgecolor=EDGE, label="High (33%)")
ax.set_xticks(xg); ax.set_xticklabels(cats)
ax.set_ylabel("Number of assets (sites)")
ax.set_title("Canada — TAM / SAM / SOM (Assets) by Exposure Scenario")
ax.legend()
plt.tight_layout()
p = OUT_DIR / "fig_CA_TAM_SAM_SOM_assets_exposure_grouped.png"
plt.savefig(p, dpi=300); plt.close()
print(f"[Saved] {p.resolve()}")

# One-time revenue (USD millions)
tam_L_M, sam_L_M, som_L_M = np.array([tam_L, sam_L, som_L]) * ASP / 1e6
tam_B_M, sam_B_M, som_B_M = np.array([tam_B, sam_B, som_B]) * ASP / 1e6
tam_H_M, sam_H_M, som_H_M = np.array([tam_H, sam_H, som_H]) * ASP / 1e6

fig, ax = plt.subplots(figsize=(8.4, 5.2))
ax.bar(xg - w, [tam_L_M, sam_L_M, som_L_M], width=w, color=colors[0], edgecolor=EDGE, label="Low (12.3%)")
ax.bar(xg,      [tam_B_M, sam_B_M, som_B_M], width=w, color=colors[1], edgecolor=EDGE, label="Base (20%)")
ax.bar(xg + w,  [tam_H_M, sam_H_M, som_H_M], width=w, color=colors[2], edgecolor=EDGE, label="High (33%)")
ax.set_xticks(xg); ax.set_xticklabels(cats)
ax.set_ylabel("USD (millions)")
ax.set_title("Canada — TAM / SAM / SOM (One-time $) by Exposure Scenario")
ax.legend()
plt.tight_layout()
p = OUT_DIR / "fig_CA_TAM_SAM_SOM_revenue_exposure_grouped_MUSD.png"
plt.savefig(p, dpi=300); plt.close()
print(f"[Saved] {p.resolve()}")

# --- Optional console recap ---
print("\n--- Recap ---")
def _fmtM(x): return f"${x/1e6:.1f}M"
print(f"US  TAM/SAM/SOM assets: {fmt_int(US_TAM_assets)} / {fmt_int(US_SAM_assets)} / {fmt_int(US_SOM_assets)}")
print(f"CA  TAM/SAM/SOM assets: {fmt_int(CA_TAM_assets)} / {fmt_int(CA_SAM_assets)} / {fmt_int(CA_SOM_assets)}")
print(f"US  $ TAM/SAM/SOM: {_fmtM(US_TAM_rev)} / {_fmtM(US_SAM_rev)} / {_fmtM(US_SOM_rev)}")
print(f"CA  $ TAM/SAM/SOM: {_fmtM(CA_TAM_rev)} / {_fmtM(CA_SAM_rev)} / {_fmtM(CA_SOM_rev)}")


[Saved] D:\Repository\mech_431\image\fig_US_TAM_SAM_SOM_assets.png
[Saved] D:\Repository\mech_431\image\fig_CA_TAM_SAM_SOM_assets.png
[Saved] D:\Repository\mech_431\image\fig_combined_TAM_SAM_SOM_assets_stacked.png
[Saved] D:\Repository\mech_431\image\fig_combined_TAM_SAM_SOM_revenue_stacked.png
[Saved] D:\Repository\mech_431\image\fig_compare_US_vs_CA_TAM_assets.png
[Saved] D:\Repository\mech_431\image\fig_compare_US_vs_CA_SAM_assets.png
[Saved] D:\Repository\mech_431\image\fig_compare_US_vs_CA_SOM_assets.png
[Saved] D:\Repository\mech_431\image\fig_compare_US_vs_CA_TAM_revenue.png
[Saved] D:\Repository\mech_431\image\fig_compare_US_vs_CA_SAM_revenue.png
[Saved] D:\Repository\mech_431\image\fig_compare_US_vs_CA_SOM_revenue.png
[Saved] D:\Repository\mech_431\image\fig_sensitivity_US_TAM_vs_exposure.png
[Saved] D:\Repository\mech_431\image\fig_sensitivity_SOM_vs_installs_per_week.png


C:\Users\gyang\AppData\Local\Temp\ipykernel_36812\796656168.py:349: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()


[Saved] D:\Repository\mech_431\image\fig_TAM_SAM_SOM_SUB_rev_single_zoom_MUSD.png
[Saved] D:\Repository\mech_431\image\fig_CA_TAM_SAM_SOM_assets_exposure_grouped.png
[Saved] D:\Repository\mech_431\image\fig_CA_TAM_SAM_SOM_revenue_exposure_grouped_MUSD.png

--- Recap ---
US  TAM/SAM/SOM assets: 148,500 / 65,340 / 2,030
CA  TAM/SAM/SOM assets: 11,340 / 4,536 / 508
US  $ TAM/SAM/SOM: $668.2M / $294.0M / $9.1M
CA  $ TAM/SAM/SOM: $51.0M / $20.4M / $2.3M


In [19]:
# Force a fixed output folder (Windows absolute path)
from pathlib import Path
OUT_DIR = Path(r"D:\Repository\mech_431\image")
OUT_DIR.mkdir(parents=True, exist_ok=True)

# === NEW: One zoom frame for SOM + Subscription (USD millions) ===
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from matplotlib.patches import Rectangle, ConnectionPatch

def fmt_money_M(v):  # value already in millions
    return f"${v:.1f}M"

# 3-yr cumulative subscription revenue split US/CA (millions)
US_active_subs = np.cumsum(np.array(US_installs_by_year)) * SERVICE_ATTACH
CA_active_subs = np.cumsum(np.array(CA_installs_by_year)) * SERVICE_ATTACH
US_subs_rev_total = float((US_active_subs * SERVICE_FEE).sum()) / 1e6
CA_subs_rev_total = float((CA_active_subs * SERVICE_FEE).sum()) / 1e6

labels4 = ["TAM", "SAM", "SOM", "Subscription"]
us_M = np.array([US_TAM_rev, US_SAM_rev, US_SOM_rev, US_subs_rev_total*1e6], float) / 1e6
ca_M = np.array([CA_TAM_rev, CA_SAM_rev, CA_SOM_rev, CA_subs_rev_total*1e6], float) / 1e6
tot_M = us_M + ca_M

fig, ax = plt.subplots(figsize=(9.2, 5.6))
x = np.arange(4)

# main stacked bars
ax.bar(x, us_M, color=C_US, edgecolor=EDGE, label="United States")
ax.bar(x, ca_M, bottom=us_M, color=C_CA, edgecolor=EDGE, label="Canada")
add_value_labels(ax, x, tot_M, fmt_money_M)

ax.set_xticks(x); ax.set_xticklabels(labels4)
ax.set_ylabel("USD (millions)")
ax.set_title("One-time & Subscription Revenue (USD millions)")
ax.legend()

# ---------- bounding box over SOM & Subscription on the main axes ----------
x0, x1 = 1.6, 3.4
y0, y1 = 0, max(tot_M[2], tot_M[3]) * 1.15
zoom_box = Rectangle((x0, y0), x1-x0, y1-y0, fill=False, lw=1.2, ec=EDGE)
ax.add_patch(zoom_box)

# ---------- single inset showing SOM + Subscription ----------
axins = inset_axes(
    ax,
    width="43.5%", height="40%",                 # smaller inset
    bbox_to_anchor=(0.52, 0.42, 1, 1),   # (x0, y0, w, h) in ax.transAxes (0–1)
    bbox_transform=ax.transAxes,
    loc="lower left",
    borderpad=0.0
)
# re-plot only SOM & Subscription (indices 2 and 3) with compact x=[0,1]
xin = np.array([0, 1])
us_in = np.array([us_M[2], us_M[3]])
ca_in = np.array([ca_M[2], ca_M[3]])
axins.bar(xin, us_in, color=C_US, edgecolor=EDGE)
axins.bar(xin, ca_in, bottom=us_in, color=C_CA, edgecolor=EDGE)
axins.set_xticks(xin); axins.set_xticklabels(["SOM", "Subscription"])
axins.set_ylim(0, max(tot_M[2], tot_M[3]) * 1.25)
axins.tick_params(axis='y', labelsize=8)

# connectors from main-axes zoom box to inset corners
con1 = ConnectionPatch(xyA=(x0, y1), coordsA=ax.transData, xyB=(0, 0), coordsB=axins.transAxes,
                       lw=1.0, color=EDGE)
con2 = ConnectionPatch(xyA=(x1, y1), coordsA=ax.transData, xyB=(1, 0), coordsB=axins.transAxes,
                       lw=1.0, color=EDGE)
ax.add_artist(con1); ax.add_artist(con2)

plt.tight_layout()
p = OUT_DIR / "fig_TAM_SAM_SOM_SUB_rev_single_zoom_MUSD.png"
plt.savefig(p, dpi=300); plt.close()
print(f"[Saved] {p.resolve()}")



C:\Users\gyang\AppData\Local\Temp\ipykernel_36812\1454007499.py:69: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()


[Saved] D:\Repository\mech_431\image\fig_TAM_SAM_SOM_SUB_rev_single_zoom_MUSD.png


In [25]:
# =========================
# Canada — TAM/SAM/SOM (Low/Base/High) with IN-BAR labels + SOM zoom
# =========================
CA_EXPOSURE_LOW  = 0.123
CA_EXPOSURE_BASE = 0.20
CA_EXPOSURE_HIGH = 0.33

def ca_tiers_assets(exposure):
    tam = CA_SITES * exposure
    sam = tam * CA_WEST_SHARE * GEN1_FIT
    cap_total = NA_CREWS * INSTALLS_PER_WEEK * WEEKS_PER_YEAR
    ca_cap = cap_total * CA_CAPACITY_SHARE
    som, _ = som_over_3yrs(sam, ca_cap, ADOPTION_RAMP)
    return tam, sam, som

tam_L, sam_L, som_L = ca_tiers_assets(CA_EXPOSURE_LOW)
tam_B, sam_B, som_B = ca_tiers_assets(CA_EXPOSURE_BASE)
tam_H, sam_H, som_H = ca_tiers_assets(CA_EXPOSURE_HIGH)

cats = ["TAM", "SAM", "SOM"]
xg = np.arange(len(cats))
w  = 0.26
colors = [C_CA_light, C_CA, "#8C1D24"]  # low / base / high

def _bar_with_inlabels_and_zoom(values_triplets, ylabel, title, out_name, value_fmt, ypad=0.0, zoom_ylim_scale=1.35):
    """values_triplets = ([tam_L,B,H], [sam_L,B,H], [som_L,B,H]) OR same scaled to $M"""
    fig, ax = plt.subplots(figsize=(8.6, 5.2))

    # group values by scenario (Low/Base/High)
    L_vals = [values_triplets[0][0], values_triplets[1][0], values_triplets[2][0]]
    B_vals = [values_triplets[0][1], values_triplets[1][1], values_triplets[2][1]]
    H_vals = [values_triplets[0][2], values_triplets[1][2], values_triplets[2][2]]

    bars_L = ax.bar(xg - w, L_vals, width=w, color=colors[0], edgecolor=EDGE, label="Low (12.3%)")
    bars_B = ax.bar(xg,      B_vals, width=w, color=colors[1], edgecolor=EDGE, label="Base (20%)")
    bars_H = ax.bar(xg + w,  H_vals, width=w, color=colors[2], edgecolor=EDGE, label="High (33%)")

    # In-bar labels (centered)
    try:
        ax.bar_label(bars_L, labels=[value_fmt(v) for v in L_vals], label_type='center', fontsize=9)
        ax.bar_label(bars_B, labels=[value_fmt(v) for v in B_vals], label_type='center', fontsize=9)
        ax.bar_label(bars_H, labels=[value_fmt(v) for v in H_vals], label_type='center', fontsize=9)
    except Exception:
        # fallback: manual text
        for bars, vals in [(bars_L, L_vals), (bars_B, B_vals), (bars_H, H_vals)]:
            for rect, v in zip(bars, vals):
                ax.text(rect.get_x() + rect.get_width()/2, rect.get_height()/2,
                        value_fmt(v), ha='center', va='center', fontsize=9, color='black')

    ax.set_xticks(xg); ax.set_xticklabels(cats)
    ax.set_ylabel(ylabel)
    ax.set_title(title)
    ax.legend()

    # ---- Zoom: focus only the SOM group (index 2), all three scenarios side-by-side
    # Bounding box over SOM triplet on main axes (data coords)
    # SOM bars are centered near x=2-w, 2, 2+w
    x0 = 2 - 1.3*w
    x1 = 2 + 1.3*w
    som_triplet_max = max(L_vals[2], B_vals[2], H_vals[2])
    y0 = 0
    y1 = som_triplet_max * zoom_ylim_scale + ypad
    zoom_box = Rectangle((x0, y0), x1 - x0, y1 - y0, fill=False, lw=1.2, ec=EDGE)
    ax.add_patch(zoom_box)

    # Inset (move/resize by tweaking bbox_to_anchor)
    from mpl_toolkits.axes_grid1.inset_locator import inset_axes
    axins = inset_axes(
        ax,
        width="30%", height="30%",
        bbox_to_anchor=(0.65, 0.4, 1, 1),  # (x0,y0,w,h) in axes coords
        bbox_transform=ax.transAxes,
        loc="lower left",
        borderpad=0.0
    )

    xin = np.array([0, 1, 2])  # Low, Base, High
    vals_in = np.array([L_vals[2], B_vals[2], H_vals[2]])
    cols_in = [colors[0], colors[1], colors[2]]
    axins.bar(xin, vals_in, width=0.6, color=cols_in, edgecolor=EDGE)
    axins.set_xticks(xin); axins.set_xticklabels(["Low", "Base", "High"])
    axins.set_ylim(0, som_triplet_max * zoom_ylim_scale + ypad)
    axins.tick_params(axis='y', labelsize=8)

    plt.tight_layout()
    p = OUT_DIR / out_name
    plt.savefig(p, dpi=300); plt.close()
    print(f"[Saved] {p.resolve()}")

# ---------- ASSETS (sites) ----------
assets_triplets = (
    [tam_L, tam_B, tam_H],
    [sam_L, sam_B, sam_H],
    [som_L, som_B, som_H],  # note: SOM is capacity-capped; identical across scenarios
)
_bar_with_inlabels_and_zoom(
    values_triplets=assets_triplets,
    ylabel="Number of assets (sites)",
    title="Canada — TAM / SAM / SOM (Assets) by Exposure Scenario",
    out_name="fig_CA_TAM_SAM_SOM_assets_exposure_grouped_inbar_zoom.png",
    value_fmt=lambda v: f"{int(round(v)):,}",
    zoom_ylim_scale=1.35
)

# ---------- ONE-TIME REVENUE (USD millions) ----------
tam_L_M, sam_L_M, som_L_M = np.array([tam_L, sam_L, som_L]) * ASP / 1e6
tam_B_M, sam_B_M, som_B_M = np.array([tam_B, sam_B, som_B]) * ASP / 1e6
tam_H_M, sam_H_M, som_H_M = np.array([tam_H, sam_H, som_H]) * ASP / 1e6

revenue_triplets = (
    [tam_L_M, tam_B_M, tam_H_M],
    [sam_L_M, sam_B_M, sam_H_M],
    [som_L_M, som_B_M, som_H_M],
)
_bar_with_inlabels_and_zoom(
    values_triplets=revenue_triplets,
    ylabel="USD (millions)",
    title="Canada — TAM / SAM / SOM (One-time $) by Exposure Scenario",
    out_name="fig_CA_TAM_SAM_SOM_revenue_exposure_grouped_MUSD_inbar_zoom.png",
    value_fmt=lambda v: f"${v:.1f}M",
    zoom_ylim_scale=1.35
)


C:\Users\gyang\AppData\Local\Temp\ipykernel_36812\3914198622.py:85: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()


[Saved] D:\Repository\mech_431\image\fig_CA_TAM_SAM_SOM_assets_exposure_grouped_inbar_zoom.png


C:\Users\gyang\AppData\Local\Temp\ipykernel_36812\3914198622.py:85: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()


[Saved] D:\Repository\mech_431\image\fig_CA_TAM_SAM_SOM_revenue_exposure_grouped_MUSD_inbar_zoom.png
